Libaries

In [15]:
# import the necessary libraries you need for your analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

#brew install cmake
#pip install xgboost==1.6.1

RSEED = 42

Reading data

In [3]:
df = pd.read_csv('data/2_data.csv')

Processing data

In [4]:
#replace state --> failed == 0 and successfull == 1
df['state'] = df['state'].replace('failed', 0).replace('successful', 1).astype(int)

#change dtype for duration from object into int
df[['duration', 'duration_days_weeks', 'duration_time']] = df['duration'].str.split(' ', expand=True)
df = df.drop('duration_days_weeks', axis = 1).drop('duration_time', axis = 1)
df.duration = df.duration.astype(int)

Defining final data frame

In [5]:
df.columns
df_final = df[['staff_pick','category_main','category_sub','duration','description_length', 'name_length', 'goal_usd','state']]
#df_final[df_final['category_sub'] == "Misc"]
df_final
df_final.description_length.fillna(0, inplace=True)

/Users/henryklange/neuefische/ds-ml-project-kickstarter/.venv/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


Replacing target with 0 and 1 and get dummies for categorial features

In [6]:
df_final['state'] = df_final['state'].replace({'failed': 0, 'successful': 1})
df_final = pd.get_dummies(df_final, drop_first = True)

y = df_final["state"]
X = df_final.drop("state", axis=1)

scaling = StandardScaler()
X_scaled = scaling.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns = list(X.columns))
X_scaled.head()

/var/folders/s_/ldsbp9yx77z3v3wtf2prgk740000gn/T/ipykernel_97058/1910993305.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['state'] = df_final['state'].replace({'failed': 0, 'successful': 1})


,staff_pick,duration,description_length,name_length,goal_usd,category_main_comics,category_main_crafts,category_main_dance,category_main_design,category_main_fashion,...,category_sub_wearables,category_sub_weaving,category_sub_web,category_sub_webcomics,category_sub_webseries,category_sub_woodworking,category_sub_workshops,category_sub_world music,category_sub_young adult,category_sub_zines
0,-0.383402,-0.208592,0.461834,-0.751669,-0.036092,-0.19773,-0.182523,-0.134647,-0.189923,-0.235358,...,-0.083067,-0.023842,-0.147519,-0.068405,-0.119498,-0.085067,-0.031264,-0.11234,-0.070087,-0.052018
1,-0.383402,-0.208592,-0.185165,1.031313,-0.023491,-0.19773,-0.182523,-0.134647,-0.189923,-0.235358,...,-0.083067,-0.023842,-0.147519,-0.068405,-0.119498,-0.085067,-0.031264,-0.11234,-0.070087,-0.052018
2,-0.383402,2.259864,0.766305,-0.051212,-0.026046,-0.19773,-0.182523,-0.134647,-0.189923,-0.235358,...,-0.083067,-0.023842,-0.147519,-0.068405,-0.119498,-0.085067,-0.031264,-0.11234,-0.070087,-0.052018
3,-0.383402,-0.208592,0.309599,0.330855,-0.034472,-0.19773,-0.182523,-0.134647,-0.189923,-0.235358,...,-0.083067,-0.023842,-0.147519,-0.068405,-0.119498,-0.085067,-0.031264,-0.11234,-0.070087,-0.052018
4,-0.383402,-0.208592,0.804363,-0.242246,-0.033842,-0.19773,-0.182523,-0.134647,-0.189923,-0.235358,...,-0.083067,-0.023842,-0.147519,-0.068405,-0.119498,-0.085067,-0.031264,-0.11234,-0.070087,-0.052018


Test_train-Split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, stratify = y, random_state=RSEED)

Logistic Regression Fitting without Tuning

In [8]:
LogReg = LogisticRegression(max_iter=500)
LogReg.fit(X_train, y_train)

LogisticRegression(max_iter=500)

Performance of Logistic Regression on test data

In [9]:
y_pred = LogReg.predict(X_test)
y_pred_test =  LogReg.predict(X_train)

print("Performance on test set: ",LogReg.score(X_test, y_test))
print("Performance on train set: ",LogReg.score(X_train, y_train))


print(classification_report(y_test,y_pred))

Performance on test set:  0.7888744230086401
Performance on train set:  0.7886044786718639
              precision    recall  f1-score   support

           0       0.73      0.82      0.77     18550
           1       0.85      0.76      0.80     23695

    accuracy                           0.79     42245
   macro avg       0.79      0.79      0.79     42245
weighted avg       0.80      0.79      0.79     42245



Gridsearch for Logistic Regression

In [10]:
# Hyperparameter grid



param_grid = {
    'max_iter': [500,1000,2000,10000],
    'penalty':["l1","l2"],
    'fit_intercept': [True, False],
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
}

# Estimator for use in random search
estimator = LogisticRegression(random_state = RSEED)

# Create the random search model
rs = GridSearchCV(estimator, param_grid, scoring= "accuracy", verbose = 5, n_jobs=-1)

# Fit 
rs.fit(X_train, y_train)

Fitting 5 folds for each of 112 candidates, totalling 560 fits
[CV 3/5] END C=0.001, fit_intercept=True, max_iter=500, penalty=l1;, score=nan total time=   0.2s
[CV 5/5] END C=0.001, fit_intercept=True, max_iter=500, penalty=l1;, score=nan total time=   0.3s
[CV 1/5] END C=0.001, fit_intercept=True, max_iter=500, penalty=l1;, score=nan total time=   0.5s
[CV 4/5] END C=0.001, fit_intercept=True, max_iter=500, penalty=l1;, score=nan total time=   0.3s
[CV 2/5] END C=0.001, fit_intercept=True, max_iter=500, penalty=l1;, score=nan total time=   0.5s
[CV 1/5] END C=0.001, fit_intercept=True, max_iter=1000, penalty=l1;, score=nan total time=   0.1s
[CV 2/5] END C=0.001, fit_intercept=True, max_iter=1000, penalty=l1;, score=nan total time=   0.1s
[CV 3/5] END C=0.001, fit_intercept=True, max_iter=1000, penalty=l1;, score=nan total time=   0.1s
[CV 4/5] END C=0.001, fit_intercept=True, max_iter=1000, penalty=l1;, score=nan total time=   0.1s
[CV 5/5] END C=0.001, fit_intercept=True, max_iter=

/Users/henryklange/neuefische/ds-ml-project-kickstarter/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 5/5] END C=1, fit_intercept=True, max_iter=500, penalty=l2;, score=0.785 total time=  25.8s
[CV 1/5] END C=1, fit_intercept=True, max_iter=10000, penalty=l1;, score=nan total time=   0.1s
[CV 2/5] END C=1, fit_intercept=True, max_iter=10000, penalty=l1;, score=nan total time=   0.1s
[CV 3/5] END C=1, fit_intercept=True, max_iter=10000, penalty=l1;, score=nan total time=   0.1s
[CV 4/5] END C=1, fit_intercept=True, max_iter=10000, penalty=l1;, score=nan total time=   0.1s
[CV 5/5] END C=1, fit_intercept=True, max_iter=10000, penalty=l1;, score=nan total time=   0.1s
[CV 4/5] END C=1, fit_intercept=True, max_iter=1000, penalty=l2;, score=0.788 total time=  20.4s
[CV 1/5] END C=1, fit_intercept=True, max_iter=2000, penalty=l2;, score=0.790 total time=  16.8s
[CV 2/5] END C=1, fit_intercept=True, max_iter=2000, penalty=l2;, score=0.788 total time=  18.2s
[CV 3/5] END C=1, fit_intercept=True, max_iter=2000, penalty=l2;, score=0.788 total time=  19.2s
[CV 5/5] END C=1, fit_intercept=True

/Users/henryklange/neuefische/ds-ml-project-kickstarter/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/5] END C=10, fit_intercept=True, max_iter=500, penalty=l2;, score=0.791 total time=  24.8s


...

[CV 4/5] END C=10, fit_intercept=True, max_iter=500, penalty=l2;, score=0.789 total time=  24.4s


/Users/henryklange/neuefische/ds-ml-project-kickstarter/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 5/5] END C=10, fit_intercept=True, max_iter=500, penalty=l2;, score=0.786 total time=  24.6s
[CV 2/5] END C=10, fit_intercept=True, max_iter=1000, penalty=l2;, score=0.789 total time=  35.1s
[CV 1/5] END C=10, fit_intercept=True, max_iter=1000, penalty=l2;, score=0.791 total time=  39.3s
[CV 3/5] END C=10, fit_intercept=True, max_iter=1000, penalty=l2;, score=0.788 total time=  50.2s
[CV 1/5] END C=10, fit_intercept=True, max_iter=10000, penalty=l1;, score=nan total time=   0.1s
[CV 2/5] END C=10, fit_intercept=True, max_iter=10000, penalty=l1;, score=nan total time=   0.1s
[CV 3/5] END C=10, fit_intercept=True, max_iter=10000, penalty=l1;, score=nan total time=   0.1s
[CV 4/5] END C=10, fit_intercept=True, max_iter=10000, penalty=l1;, score=nan total time=   0.1s
[CV 5/5] END C=10, fit_intercept=True, max_iter=10000, penalty=l1;, score=nan total time=   0.1s
[CV 4/5] END C=10, fit_intercept=True, max_iter=1000, penalty=l2;, score=0.789 total time=  38.0s
[CV 2/5] END C=10, fit_int

/Users/henryklange/neuefische/ds-ml-project-kickstarter/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
280 fits failed out of a total of 560.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
280 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/henryklange/neuefische/ds-ml-project-kickstarter/.venv/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/henryklange/neuefische/ds-ml-project-kickstarter/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.du

GridSearchCV(estimator=LogisticRegression(random_state=42), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                         'fit_intercept': [True, False],
                         'max_iter': [500, 1000, 2000, 10000],
                         'penalty': ['l1', 'l2']},
             scoring='accuracy', verbose=5)

Printing best parameters after tuning

In [11]:
print("tuned hpyerparameters :(best parameters) ",rs.best_params_)
print("accuracy :",rs.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 10, 'fit_intercept': True, 'max_iter': 500, 'penalty': 'l2'}
accuracy : 0.7886281277075176


Performance of Logistic Regression after tuning

In [13]:
logreg_best = rs.best_estimator_

y_pred_best = logreg_best.predict(X_test)


#print("Performance on train set: ",LogReg.score(X_train, y_train))
#print("Performance on test set: ",LogReg.score(X_test, y_pred_best))


print(classification_report(y_test,y_pred_best))

              precision    recall  f1-score   support

           0       0.73      0.82      0.77     18550
           1       0.84      0.77      0.80     23695

    accuracy                           0.79     42245
   macro avg       0.79      0.79      0.79     42245
weighted avg       0.80      0.79      0.79     42245



XGBOOST

Defining XGBClassifier and fitting it

In [16]:
boost = XGBClassifier()
boost.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

Predicting y with untuned model 

In [17]:
y_pred_boost = boost.predict(X_test)

In [20]:
print("Performance on test set: ",accuracy_score(y_test, y_pred_boost))
print(classification_report(y_test, y_pred_boost))

Performance on test set:  0.8001420286424429
              precision    recall  f1-score   support

           0       0.75      0.81      0.78     18550
           1       0.84      0.79      0.82     23695

    accuracy                           0.80     42245
   macro avg       0.80      0.80      0.80     42245
weighted avg       0.80      0.80      0.80     42245



Defining a grid search for XGBoost with n_estimators, subsample size, min_child_weight and learning rate

In [22]:
boost2 = XGBClassifier()

param_grid_x = [ 
  {'n_estimators': [100, 200],
   'subsample': [0.7, 0.9],
   'min_child_weight': [100, 200],
   'learning_rate': [0.1,0.3]
  }
]

grid_boost2 = GridSearchCV(estimator=boost2, param_grid=param_grid_x, cv=5,scoring= "accuracy", verbose = 5, n_jobs=-1)

grid_boost2.fit(X_train, y_train)



Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 3/5] END learning_rate=0.1, min_child_weight=100, n_estimators=100, subsample=0.9;, score=0.792 total time= 2.6min
[CV 1/5] END learning_rate=0.1, min_child_weight=100, n_estimators=100, subsample=0.9;, score=0.799 total time= 2.6min
[CV 2/5] END learning_rate=0.1, min_child_weight=100, n_estimators=100, subsample=0.9;, score=0.794 total time= 2.6min
[CV 1/5] END learning_rate=0.1, min_child_weight=100, n_estimators=100, subsample=0.7;, score=0.797 total time= 2.8min
[CV 4/5] END learning_rate=0.1, min_child_weight=100, n_estimators=100, subsample=0.7;, score=0.789 total time= 2.8min
[CV 5/5] END learning_rate=0.1, min_child_weight=100, n_estimators=100, subsample=0.7;, score=0.791 total time= 2.8min
[CV 2/5] END learning_rate=0.1, min_child_weight=100, n_estimators=100, subsample=0.7;, score=0.793 total time= 2.9min
[CV 3/5] END learning_rate=0.1, min_child_weight=100, n_estimators=100, subsample=0.7;, score=0.790 total 

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_ca...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                        

getting best grid search parameters

In [25]:
grid_boost2.best_params_

{'learning_rate': 0.3,
 'min_child_weight': 100,
 'n_estimators': 200,
 'subsample': 0.9}

fitting model with best parameters and printing the scores

In [30]:
boost_best = grid_boost2.best_estimator_

y_pred_best2 = grid_boost2.predict(X_test)


#print("Performance on train set: ",LogReg.score(X_train, y_train))
print("Performance on test set: ",boost_best.score(X_test, y_test))


print(classification_report(y_test,y_pred_best2))

Performance on test set:  0.7988400994200497
              precision    recall  f1-score   support

           0       0.76      0.79      0.78     18550
           1       0.83      0.81      0.82     23695

    accuracy                           0.80     42245
   macro avg       0.80      0.80      0.80     42245
weighted avg       0.80      0.80      0.80     42245



running another gridsearch with max_depth and more learning rates, dropping  n_estimators

In [26]:
boost3 = XGBClassifier()

param_grid_x = [ 
  {'max_depth': [5, 10, 15],
   'subsample': [0.7, 0.9],
   'min_child_weight': [50, 100],
   'learning_rate': [0.1, 0.5, 0.75, 0.99]
  }
]

grid_boost3 = GridSearchCV(estimator=boost3, param_grid=param_grid_x, cv=5, scoring= "accuracy", verbose = 5, n_jobs=-1)

grid_boost3.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 2/5] END learning_rate=0.1, max_depth=5, min_child_weight=50, subsample=0.9;, score=0.792 total time= 2.0min
[CV 1/5] END learning_rate=0.1, max_depth=5, min_child_weight=50, subsample=0.9;, score=0.793 total time= 2.0min
[CV 3/5] END learning_rate=0.1, max_depth=5, min_child_weight=50, subsample=0.9;, score=0.789 total time= 2.0min
[CV 1/5] END learning_rate=0.1, max_depth=5, min_child_weight=50, subsample=0.7;, score=0.795 total time= 2.2min
[CV 5/5] END learning_rate=0.1, max_depth=5, min_child_weight=50, subsample=0.7;, score=0.787 total time= 2.2min
[CV 2/5] END learning_rate=0.1, max_depth=5, min_child_weight=50, subsample=0.7;, score=0.789 total time= 2.2min
[CV 3/5] END learning_rate=0.1, max_depth=5, min_child_weight=50, subsample=0.7;, score=0.789 total time= 2.2min
[CV 4/5] END learning_rate=0.1, max_depth=5, min_child_weight=50, subsample=0.7;, score=0.788 total time= 2.2min
[CV 5/5] END learning_rate=0.1, ma

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     gamma=None, gpu_id=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_bin=None,
                                     max_ca...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                        

getting best parameters for second XGBoost gridsearch, fitting model, predict y values and get scoring

In [31]:
boost_best3 = grid_boost3.best_estimator_

y_pred_best3 = grid_boost3.predict(X_test)


print("Performance on train set: ",boost_best3.score(X_train, y_train))
print("Performance on test set: ",boost_best3.score(X_test, y_test))


print(classification_report(y_test,y_pred_best3))

Performance on train set:  0.8080152129657393
Performance on test set:  0.8018937152325719
              precision    recall  f1-score   support

           0       0.76      0.80      0.78     18550
           1       0.83      0.81      0.82     23695

    accuracy                           0.80     42245
   macro avg       0.80      0.80      0.80     42245
weighted avg       0.80      0.80      0.80     42245



In [40]:
grid_boost3.best_params_

{'learning_rate': 0.5,
 'max_depth': 5,
 'min_child_weight': 50,
 'subsample': 0.9}

Dropping category main and running model again

In [46]:
df_final2 = df[['staff_pick','category_sub','duration','description_length', 'name_length', 'goal_usd','state']]
df_final2.description_length.fillna(0, inplace=True)
df_final2['state'] = df_final2['state'].replace({'failed': 0, 'successful': 1})
df_final2 = pd.get_dummies(df_final2, drop_first = True)

y2 = df_final2["state"]
X2 = df_final2.drop("state", axis=1)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, stratify = y2, random_state=RSEED)



/Users/henryklange/neuefische/ds-ml-project-kickstarter/.venv/lib/python3.9/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/var/folders/s_/ldsbp9yx77z3v3wtf2prgk740000gn/T/ipykernel_97058/3539301915.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final2['state'] = df_final2['state'].replace({'failed': 0, 'successful': 1})


In [48]:
col = ['duration','description_length', 'name_length', 'goal_usd']
scaling2 = StandardScaler()
X_train_scaled = scaling2.fit_transform(X_train2[col])
X_test_scaled = scaling2.transform(X_test2[col])

X_train_preprocessed = np.concatenate([X_train_scaled, X_train2.drop(col , axis=1)], axis= 1)
X_test_preprocessed = np.concatenate([X_test_scaled, X_test2.drop(col , axis=1)], axis = 1)

In [49]:
#fitting XGBoost with final params and dropped main category

boost_final = XGBClassifier(learning_rate = 0.5, max_depth = 5, min_child_weight = 50, subsample = 0.9)

boost_final.fit(X_train_preprocessed, y_train2)

y_pred_final = boost_final.predict(X_test_preprocessed)

print("Performance on train set: ",boost_final.score(X_train_preprocessed, y_train2))
print("Performance on test set: ",boost_final.score(X_test_preprocessed, y_test2))


print(classification_report(y_test2,y_pred_final))

Performance on train set:  0.8065554626225007
Performance on test set:  0.8
              precision    recall  f1-score   support

           0       0.76      0.80      0.78     18550
           1       0.84      0.80      0.82     23695

    accuracy                           0.80     42245
   macro avg       0.80      0.80      0.80     42245
weighted avg       0.80      0.80      0.80     42245

